## wikidata id disambiguation
This code processes a Wikidata ID generated by a Large Language Model (LLM) to validate and refine it. It accesses the Wikidata SPARQL endpoint to retrieve a list of potential matches for the given label. Using Claude for disambiguation, the system incorporates contextual information, such as coordinates previously extracted by the LLM, and specifies that the entities of interest are geospatial features. These include countries, cities, villages, buildings, museums, and cultural heritage sites. Once the correct entity is identified, the disambiguated Wikidata ID is substituted back into the original TRiG file.

In [4]:
import os
import re
import requests
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass
from anthropic import Anthropic
from SPARQLWrapper import SPARQLWrapper, JSON

@dataclass
class Entity:
    wikidata_id: str
    label: str
    coordinates: Optional[str]
    entity_type: str

@dataclass
class WikidataCandidate:
    id: str
    label: str
    description: str

@dataclass
class DisambiguationResult:
    file: str
    original_id: str
    label: str
    coordinates: Optional[str]
    disambiguated_id: Optional[str]

class WikidataDisambiguator:
    def __init__(self, api_key: str):
        self.client = Anthropic(api_key=api_key)
        self.sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        
    def validate_wikidata_id(self, wikidata_id: str, expected_label: str) -> Tuple[bool, Optional[str]]:
        """Validate if a Wikidata ID matches the expected label."""
        wikidata_id = wikidata_id.lstrip("Q")
        query = f"""
        SELECT ?entityLabel WHERE {{
            BIND(wd:Q{wikidata_id} AS ?entity)
            ?entity rdfs:label ?entityLabel.
            FILTER(LANG(?entityLabel) IN ("en", "it"))
        }}
        """
        self.sparql.setQuery(query)
        self.sparql.setReturnFormat(JSON)

        try:
            results = self.sparql.query().convert()
            for result in results["results"]["bindings"]:
                label = result["entityLabel"]["value"]
                if expected_label.lower() == label.lower():
                    return True, label
            return False, None
        except Exception as e:
            print(f"SPARQL Error: {e}")
            return False, None

    def search_wikidata_candidates(self, label: str) -> List[WikidataCandidate]:
        """Search for potential Wikidata matches for a given label."""
        url = "https://www.wikidata.org/w/api.php"
        params = {
            "action": "wbsearchentities",
            "search": label,
            "language": "en",
            "uselang": "it",
            "format": "json",
            "type": "item"
        }

        try:
            response = requests.get(url, params=params)
            data = response.json()
            return [
                WikidataCandidate(
                    id=result["id"].lstrip('Q'),
                    label=result["label"],
                    description=result.get("description", "")
                )
                for result in data.get("search", [])
            ]
        except Exception as e:
            print(f"API Error: {e}")
            return []

    def gpt_disambiguation(
        self, 
        original_id: str, 
        original_label: str, 
        candidates: List[WikidataCandidate],
        coordinates: Optional[str],
        entity_type: str
    ) -> Optional[str]:
        """Use Claude to disambiguate between multiple Wikidata candidates."""
        prompt = self._build_gpt_prompt(
            original_id, original_label, candidates, coordinates, entity_type
        )

        try:
            response = self.client.messages.create(
                model="claude-3-5-sonnet-latest",
                max_tokens=1000,
                messages=[{
                    "role": "user", 
                    "content": prompt
                }]
            )
            return response.content[0].text.strip().lstrip('Q')
        except Exception as e:
            print(f"Claude API Error: {e}")
            return None

    def _build_gpt_prompt(
        self, 
        original_id: str, 
        original_label: str, 
        candidates: List[WikidataCandidate],
        coordinates: Optional[str],
        entity_type: str
    ) -> str:
        """Build the prompt for Claude disambiguation."""
        prompt = f"""
        The Wikidata ID Q{original_id} with label '{original_label}' needs to be validated.
        Context:
        - Entity type: {entity_type}
        - Coordinates: {coordinates}

        Here are some alternative candidates with their descriptions:
        """
        for candidate in candidates:
            prompt += f"- {candidate.label} (Q{candidate.id}): {candidate.description}\n"

        prompt += "\nConsidering the context provided and considering that these entities all refer to geospatial data (i.e. cultural heritage sites, buildings, cities), which candidate is the correct match? Provide only the Wikidata ID number without the Q prefix."
        return prompt

class TrigFileProcessor:
    @staticmethod
    def extract_info(file_path: str) -> List[Entity]:
        """Extract entity information from a TRIG file."""
        entities = []

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

                wikidata_pattern = r'<https://www\.wikidata\.org/wiki/Q(\d+)>'
                wikidata_matches = re.finditer(wikidata_pattern, content)

                for wikidata_match in wikidata_matches:
                    entity = TrigFileProcessor._extract_entity(content, wikidata_match.group(1))
                    if entity:
                        entities.append(entity)

        except Exception as e:
            print(f"Error reading file {file_path}: {e}")

        return entities

    @staticmethod
    def _extract_entity(content: str, wikidata_id: str) -> Optional[Entity]:
        """Extract a single entity's information from TRIG content."""
        label_pattern = rf'<https://www\.wikidata\.org/wiki/Q{wikidata_id}>\s+rdfs:label\s+"([^"]+)"'
        label_match = re.search(label_pattern, content)

        if not label_match:
            value_pattern = r'rdf:value\s+"([^"]+)"'
            label_match = re.search(value_pattern, content)

        if not label_match:
            return None

        coordinates_pattern = r'crm:P168_place_is_defined_by\s+"([^"]+)"'
        coordinates_match = re.search(coordinates_pattern, content)

        return Entity(
            wikidata_id=wikidata_id,
            label=label_match.group(1),
            coordinates=coordinates_match.group(1) if coordinates_match else None,
            entity_type="Location"
        )

    @staticmethod
    def update_file(file_path: str, original_id: str, new_id: str) -> None:
        """Update Wikidata IDs in a TRIG file."""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

            updated_content = content.replace(
                f'<https://www.wikidata.org/wiki/Q{original_id}>',
                f'<https://www.wikidata.org/wiki/Q{new_id}>'
            )

            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(updated_content)

            print(f"Updated Wikidata ID from Q{original_id} to Q{new_id} in {file_path}")

        except Exception as e:
            print(f"Error updating file {file_path}: {e}")

def main():
    # Initialize the disambiguator with your API key
    api_key = ''
    if not api_key:
        raise ValueError("Please set the ANTHROPIC_API_KEY environment variable")
        
    disambiguator = WikidataDisambiguator(api_key=api_key)
    
    # Set your input folder path
    folder_path = "./rdf_output"
    results: List[DisambiguationResult] = []

    # Process each TRIG file
    for filename in os.listdir(folder_path):
        if not filename.endswith('.trig'):
            continue

        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")

        try:
            entities = TrigFileProcessor.extract_info(file_path)

            for entity in entities:
                # Validate and disambiguate
                is_valid, _ = disambiguator.validate_wikidata_id(entity.wikidata_id, entity.label)
                
                disambiguated_id = None
                if not is_valid:
                    candidates = disambiguator.search_wikidata_candidates(entity.label)
                    if candidates:
                        disambiguated_id = disambiguator.gpt_disambiguation(
                            entity.wikidata_id,
                            entity.label,
                            candidates,
                            entity.coordinates,
                            entity.entity_type
                        )

                # Store results
                results.append(DisambiguationResult(
                    file=filename,
                    original_id=entity.wikidata_id,
                    label=entity.label,
                    coordinates=entity.coordinates,
                    disambiguated_id=disambiguated_id
                ))

                # Update file if needed
                if disambiguated_id and disambiguated_id != entity.wikidata_id:
                    TrigFileProcessor.update_file(file_path, entity.wikidata_id, disambiguated_id)

        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

    # Print results
    for result in results:
        print(f"\nFile: {result.file}")
        print(f"Original ID: Q{result.original_id}")
        print(f"Label: {result.label}")
        print(f"Coordinates: {result.coordinates}")
        print(f"Disambiguated ID: {result.disambiguated_id if result.disambiguated_id else 'Not found'}")

if __name__ == "__main__":
    main()

Processing file: 10_date_1_sunday feb 18 1894.trig
Processing file: 10_date_2_monday feb 19 1894.trig
Processing file: 10_event_3e7a7f1a-5a9a-4d8b-9b7e-2d1d7f64c1e7.trig
Processing file: 10_event_3e7a7f1a-5a9a-4d8b-9b7e-2d49f5e5d6c5.trig
Processing file: 10_event_6f9a3c1a-9d9a-4f9c-bd9f-2d9d6d9d5d9d.trig
Processing file: 10_event_d8d8f7d6-9a9e-4f0c-9d0f-2c9d6d9d4d9d.trig
Processing file: 10_event_d8d8f7d6-9a9e-4f0e-9f7d-a6e6d5d6d0d4.trig
Processing file: 10_event_f8d685f2-283c-4677-a7f6-d5d9d9d9d9d9.trig
Processing file: 10_place_1_il palmerino maiano.trig
Processing file: 10_place_2_12 lungarno acciajuoli florence.trig
Processing file: 10_place_3_uffizi.trig
Updated Wikidata ID from Q127665 to Q51252 - This ID corresponds to "Palazzo degli Uffizi" which is the most appropriate match for the coordinates provided (43.7677, 11.2556) in Florence, Italy. This ID specifically refers to the physical building and art museum rather than the institution (Q28647411), a specific room (Q61776682),

KeyboardInterrupt: 

In [ ]:
import os
import re
import requests
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass
from anthropic import Anthropic

@dataclass
class Entity:
    geonames_id: str
    label: str
    coordinates: Optional[str]
    entity_type: str

@dataclass
class GeonamesCandidate:
    id: str
    label: str
    description: str
    lat: Optional[str] = None
    lng: Optional[str] = None

@dataclass
class DisambiguationResult:
    file: str
    original_id: str
    label: str
    coordinates: Optional[str]
    disambiguated_id: Optional[str]

class GeonamesDisambiguator:
    def __init__(self, api_key: str):
        self.client = Anthropic(api_key=api_key)

    def validate_geonames_id(self, geonames_id: str, expected_label: str) -> Tuple[bool, Optional[str]]:
        """Validate if a GeoNames ID matches the expected label."""
        url = f"http://api.geonames.org/getJSON?geonameId={geonames_id}&username=demo"  # Replace 'demo' with your GeoNames username
        try:
            response = requests.get(url)
            data = response.json()
            print("GeoNames API Response for Validation:", data)
            if data.get("name") and expected_label.lower() == data["name"].lower():
                return True, data["name"]
            return False, None
        except Exception as e:
            print(f"GeoNames API Error: {e}")
            return False, None

    def search_geonames_candidates(self, label: str) -> List[GeonamesCandidate]:
        """Search for potential GeoNames matches for a given label."""
        url = "http://api.geonames.org/searchJSON"
        params = {
            "q": label,
            "maxRows": 10,
            "username": "demo"  # Replace 'demo' with your GeoNames username
        }

        try:
            response = requests.get(url, params=params)
            data = response.json()
            print("GeoNames API Response for Search:", data)
            return [
                GeonamesCandidate(
                    id=result["geonameId"],
                    label=result["name"],
                    description=result.get("fcodeName", ""),
                    lat=result.get("lat"),
                    lng=result.get("lng")
                )
                for result in data.get("geonames", [])
            ]
        except Exception as e:
            print(f"GeoNames API Error: {e}")
            return []

    def gpt_disambiguation(
        self, 
        original_id: str, 
        original_label: str, 
        candidates: List[GeonamesCandidate],
        coordinates: Optional[str],
        entity_type: str
    ) -> Optional[str]:
        """Use Claude to disambiguate between multiple GeoNames candidates."""
        prompt = self._build_gpt_prompt(
            original_id, original_label, candidates, coordinates, entity_type
        )

        try:
            response = self.client.messages.create(
                model="claude-3-5-sonnet-latest",
                max_tokens=1000,
                messages=[{
                    "role": "user",
                    "content": prompt
                }]
            )
            result = response.content[0].text.strip()
            # Handle potential 'None' response
            return None if result.lower() == 'none' else result
        except Exception as e:
            print(f"Claude API Error: {e}")
            return None

    def _build_gpt_prompt(
        self, 
        original_id: str, 
        original_label: str, 
        candidates: List[GeonamesCandidate],
        coordinates: Optional[str],
        entity_type: str
    ) -> str:
        """Build the prompt for Claude disambiguation."""
        prompt = f"""Task: Match the location '{original_label}' to the correct GeoNames ID from the provided candidates.

Current Information:
- Original GeoNames ID: {original_id}
- Location Name: {original_label}
- Entity Type: {entity_type}
- Known Coordinates: {coordinates if coordinates else 'Not provided'}

Available Candidates:
"""
        for idx, candidate in enumerate(candidates, 1):
            prompt += f"{idx}. Name: {candidate.label}\n   ID: {candidate.id}\n   Type: {candidate.description}\n   Coordinates: ({candidate.lat}, {candidate.lng})\n\n"

        prompt += """Instructions:
1. Compare the original location name with the candidate names
2. Consider the geographical context (coordinates if available)
3. Consider the location type/description
4. Select the most appropriate match

Respond ONLY with the GeoNames ID of the best matching candidate. If none of the candidates are suitable, respond with 'None'.
Format your response as a single GeoNames ID number or 'None'."""

        return prompt

class TrigFileProcessor:
    @staticmethod
    def extract_info(file_path: str) -> List[Entity]:
        """Extract entity information from a TRIG file."""
        entities = []

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

                geonames_pattern = r'<https://www\.geonames\.org/(\d+)>'
                geonames_matches = re.finditer(geonames_pattern, content)

                for geonames_match in geonames_matches:
                    entity = TrigFileProcessor._extract_entity(content, geonames_match.group(1))
                    if entity:
                        entities.append(entity)

        except Exception as e:
            print(f"Error reading file {file_path}: {e}")

        return entities

    @staticmethod
    def _extract_entity(content: str, geonames_id: str) -> Optional[Entity]:
        """Extract a single entity's information from TRIG content."""
        label_pattern = rf'<https://www\.wikidata\.org/wiki/Q\d+>\s+rdfs:label\s+"([^"]+)"'
        label_match = re.search(label_pattern, content)

        if not label_match:
            value_pattern = r'rdf:value\s+"([^"]+)"'
            label_match = re.search(value_pattern, content)

        if not label_match:
            return None

        coordinates_pattern = r'crm:P168_place_is_defined_by\s+"([^"]+)"'
        coordinates_match = re.search(coordinates_pattern, content)

        return Entity(
            geonames_id=geonames_id,
            label=label_match.group(1),
            coordinates=coordinates_match.group(1) if coordinates_match else None,
            entity_type="Location"
        )

    @staticmethod
    def update_file(file_path: str, original_id: str, new_id: str) -> None:
        """Update GeoNames IDs in a TRIG file."""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

            updated_content = content.replace(
                f'<https://www.geonames.org/{original_id}>',
                f'<https://www.geonames.org/{new_id}>'
            )

            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(updated_content)

            print(f"Updated GeoNames ID from {original_id} to {new_id} in {file_path}")

        except Exception as e:
            print(f"Error updating file {file_path}: {e}")

def main():
    # Initialize the disambiguator with your API key
    api_key = ''
    if not api_key:
        raise ValueError("Please set the ANTHROPIC_API_KEY environment variable")
        
    disambiguator = GeonamesDisambiguator(api_key=api_key)
    
    # Set your input folder path
    folder_path = "./rdf_output"
    results: List[DisambiguationResult] = []

    # Process each TRIG file
    for filename in os.listdir(folder_path):
        if not filename.endswith('.trig'):
            continue

        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")

        try:
            entities = TrigFileProcessor.extract_info(file_path)

            for entity in entities:
                # Validate and disambiguate
                is_valid, _ = disambiguator.validate_geonames_id(entity.geonames_id, entity.label)
                
                disambiguated_id = None
                if not is_valid:
                    candidates = disambiguator.search_geonames_candidates(entity.label)
                    if candidates:
                        disambiguated_id = disambiguator.gpt_disambiguation(
                            entity.geonames_id,
                            entity.label,
                            candidates,
                            entity.coordinates,
                            entity.entity_type
                        )

                # Store results
                results.append(DisambiguationResult(
                    file=filename,
                    original_id=entity.geonames_id,
                    label=entity.label,
                    coordinates=entity.coordinates,
                    disambiguated_id=disambiguated_id
                ))

                # Update file if needed
                if disambiguated_id and disambiguated_id != entity.geonames_id:
                    print(f"Updating GeoNames ID in file {filename}: {entity.geonames_id} -> {disambiguated_id}")
                    TrigFileProcessor.update_file(file_path, entity.geonames_id, disambiguated_id)
                else:
                    print(f"No update needed for GeoNames ID {entity.geonames_id} in file {filename}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

    # Print results
    for result in results:
        print(f"\nFile: {result.file}")
        print(f"Original ID: {result.original_id}")
        print(f"Label: {result.label}")
        print(f"Coordinates: {result.coordinates}")
        print(f"Disambiguated ID: {result.disambiguated_id if result.disambiguated_id else 'Not found'}")

if __name__ == "__main__":
    main()

Processing file: 10_date_1_sunday feb 18 1894.trig
Processing file: 10_date_2_monday feb 19 1894.trig
Processing file: 10_event_3e7a7f1a-5a9a-4d8b-9b7e-2d1d7f64c1e7.trig
Processing file: 10_event_3e7a7f1a-5a9a-4d8b-9b7e-2d49f5e5d6c5.trig
Processing file: 10_event_6f9a3c1a-9d9a-4f9c-bd9f-2d9d6d9d5d9d.trig
Processing file: 10_event_d8d8f7d6-9a9e-4f0c-9d0f-2c9d6d9d4d9d.trig
Processing file: 10_event_d8d8f7d6-9a9e-4f0e-9f7d-a6e6d5d6d0d4.trig
Processing file: 10_event_f8d685f2-283c-4677-a7f6-d5d9d9d9d9d9.trig
Processing file: 10_place_1_il palmerino maiano.trig
GeoNames API Response for Validation: {'timezone': {'gmtOffset': 1, 'timeZoneId': 'Europe/Rome', 'dstOffset': 2}, 'asciiName': 'Stazione Lido Castel Fusano', 'astergdem': 7, 'countryId': '3175395', 'fcl': 'S', 'srtm3': 8, 'adminId2': '3169069', 'adminId3': '3169071', 'countryCode': 'IT', 'adminId4': '11396087', 'adminId5': '11396199', 'adminCodes2': {'ISO3166_2': 'RM'}, 'adminCodes1': {'ISO3166_2': '62'}, 'adminId1': '3174976', 'lat'